In [11]:
from sklearn.datasets import fetch_openml
import time
import numpy as np
from numpy import linalg as LA
from sklearn.model_selection import train_test_split
import time
import matplotlib.pyplot as plt
from cvxopt import matrix, solvers 

In [12]:
# Load the dataset
with np.load('mnist.npz') as data:
    xx = data['x_train']
    yy = data['y_train']

xx = xx.reshape(-1, 784) / 255.  # normalization of the data
yy = yy.astype(np.int64)

m = xx.shape[0]
n = xx.shape[1] + 1

X = np.concatenate((np.ones([m,1]),xx), axis=1)  # add one to each data point

cat = np.zeros([m,10])
for ind, num in enumerate(yy):
    cat[ind][num] = 1
Y = cat

# split the dataset into training and testing datasets
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.15, random_state=42)


In [13]:
dg1=1
dg2=2
num12 = sum(Y[:,dg1]) + sum(Y[:,dg2])
Y1 = Y[:,dg1];
Y2 = Y[:,dg2];
indx1 = np.where(Y1==1)[0];
indx2 = np.where(Y2==1)[0];

indx12 = np.concatenate((indx1,indx2))

X12 = X[indx12,1:]
Y12 = np.ones(len(indx12))
Y12[len(indx1):] =-1

print(X12)
print(Y12)

xdual_train, xdual_test, ydual_train, ydual_test = train_test_split(X12, Y12, test_size = 0.15, random_state=24)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[ 1.  1.  1. ... -1. -1. -1.]


In [14]:
P = matrix(np.outer(ydual_train, ydual_train) * np.dot(xdual_train, xdual_train.T))
q = matrix(-np.ones((xdual_train.shape[0], 1)))
G = matrix(-np.eye(xdual_train.shape[0]))
h = matrix(np.zeros(xdual_train.shape[0]))
A = matrix(ydual_train.reshape(1, -1))
b = matrix(np.zeros(1))

In [15]:
solvers.options['show_progress'] = False
solution = solvers.qp(P, q, G, h, A, b)

In [16]:
alphas = np.array(solution['x'])

In [17]:
w = np.sum(alphas * ydual_train[:, None] * xdual_train, axis = 0)

In [18]:
S = (alphas > 1e-4).flatten()
b = ydual_train[S] - np.dot(xdual_train[S], w)

In [19]:
b = np.mean(b)

print('Alphas: ', alphas[S])
print('w: ', w)
print('b: ', b)

Alphas:  [[4.02971814e-01]
 [4.33174455e+00]
 [7.70848814e-01]
 [2.20515070e-02]
 [3.08992595e+00]
 [8.83940191e-01]
 [5.03563984e-01]
 [2.56243805e+00]
 [2.36002691e+00]
 [6.00108581e-01]
 [2.16597141e-01]
 [7.52372953e-01]
 [7.25904878e-02]
 [3.03680589e-02]
 [5.50827106e+00]
 [8.76352160e-01]
 [2.66985131e+00]
 [1.98524057e+00]
 [8.02062138e-01]
 [4.19212349e-01]
 [1.35791630e+00]
 [8.94206084e-01]
 [1.76891740e+00]
 [5.52637783e-01]
 [3.63008987e-01]
 [1.99212599e-01]
 [8.74562249e-02]
 [4.44465163e-02]
 [1.68872327e+00]
 [5.92459769e-01]
 [2.56264040e+00]
 [2.73609864e-01]
 [2.13480170e-01]
 [1.99575219e+00]
 [2.68090319e+00]
 [2.55273538e-01]
 [9.33229376e-02]
 [1.47017468e+00]
 [1.29557402e-02]
 [6.57149993e-01]
 [4.43845416e+00]
 [8.22066508e-01]
 [3.02080739e+00]
 [1.97955558e+00]
 [5.15951092e-01]
 [5.84332739e-01]
 [6.37463348e-01]
 [1.14247365e+00]
 [1.30149500e+00]
 [4.01088812e+00]
 [8.35737614e-01]
 [6.76805629e-01]
 [5.76013714e-01]
 [5.04119391e-01]
 [5.73762738e-01]
 

In [20]:
predictions = np.sign(np.dot(xdual_test, w) + b)

accuracy = np.mean(predictions == ydual_test)

print('Accuracy: ', accuracy)


Accuracy:  0.9874015748031496
